In [3]:
%matplotlib inline
import pandas as pd
import numpy as np

from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# Data Preparation using Pandas

An initial step in statistical data analysis is the preparation of the data to be used in the analysis. In practice, ~~a little~~ ~~some~~ ~~much~~ the majority of the actual time spent on a statistical modeling project is typically devoted to importing, cleaning, validating and transforming the dataset.

This section will introduce [pandas](http://pandas.pydata.org/), an important third-party Python package for data analysis, as a tool for data preparation, and provide some general advice for what should or should not be done to data before it is analyzed.

## Introduction to pandas

**pandas** is a Python package providing fast, flexible, and expressive data structures designed to work with *relational* or *labeled* data both. It is a fundamental high-level building block for doing practical, real world data analysis in Python. 

pandas is well suited for:

- **Tabular** data with heterogeneously-typed columns, as you might find in an SQL table or Excel spreadsheet
- Ordered and unordered (not necessarily fixed-frequency) **time series** data.
- Arbitrary **matrix** data with row and column labels

Virtually any statistical dataset, labeled or unlabeled, can be converted to a pandas data structure for cleaning, transformation, and analysis.


### Key features
    
- Easy handling of **missing data**
- **Size mutability**: columns can be inserted and deleted from DataFrame and higher dimensional objects
- Automatic and explicit **data alignment**: objects can be explicitly aligned to a set of labels, or the data can be aligned automatically
- Powerful, flexible **group by functionality** to perform split-apply-combine operations on data sets
- Intelligent label-based **slicing, fancy indexing, and subsetting** of large data sets
- Intuitive **merging and joining** data sets
- Flexible **reshaping and pivoting** of data sets
- **Hierarchical labeling** of axes
- Robust **IO tools** for loading data from flat files, Excel files, databases, and HDF5
- **Time series functionality**: date range generation and frequency conversion, moving window statistics, moving window linear regressions, date shifting and lagging, etc.

### Series

A **Series** is a single vector of data (like a NumPy array) with an *index* that labels each element in the vector.

In [4]:
counts = pd.Series([632, 1638, 569, 115])
counts

0     632
1    1638
2     569
3     115
dtype: int64

If an index is not specified, a default sequence of integers is assigned as the index. A NumPy array comprises the values of the `Series`, while the index is a pandas `Index` object.

In [5]:
counts.values

array([ 632, 1638,  569,  115])

In [6]:
counts.index

Int64Index([0, 1, 2, 3], dtype='int64')

We can assign meaningful labels to the index, if they are available:

In [7]:
bacteria = pd.Series([632, 1638, 569, 115], 
    index=['Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes'])

bacteria

Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
dtype: int64

These labels can be used to refer to the values in the `Series`.

In [8]:
bacteria['Actinobacteria']

569

In [9]:
bacteria[[name.endswith('bacteria') for name in bacteria.index]]

Proteobacteria    1638
Actinobacteria     569
dtype: int64

In [10]:
[name.endswith('bacteria') for name in bacteria.index]

[False, True, True, False]

In [11]:
'Bacteroidetes' in bacteria

True

Notice that the indexing operation preserved the association between the values and the corresponding indices.

We can still use positional indexing if we wish.

In [12]:
bacteria[0]

632

We can give both the array of values and the index meaningful labels themselves:

In [13]:
bacteria.name = 'counts'
bacteria.index.name = 'phylum'
bacteria

phylum
Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
Name: counts, dtype: int64

NumPy's math functions and other operations can be applied to Series without losing the data structure.

In [14]:
np.log(bacteria)

phylum
Firmicutes        6.448889
Proteobacteria    7.401231
Actinobacteria    6.343880
Bacteroidetes     4.744932
Name: counts, dtype: float64

We can also filter according to the values in the `Series`:

In [15]:
bacteria[bacteria>1000]

phylum
Proteobacteria    1638
Name: counts, dtype: int64

A `Series` can be thought of as an ordered key-value store. In fact, we can create one from a `dict`:

In [16]:
bacteria_dict.values()

NameError: name 'bacteria_dict' is not defined

In [17]:
bacteria_dict = {'Firmicutes': 632, 'Proteobacteria': 1638, 'Actinobacteria': 569, 'Bacteroidetes': 115}
bact = pd.Series(bacteria_dict)

In [18]:
bact

Actinobacteria     569
Bacteroidetes      115
Firmicutes         632
Proteobacteria    1638
dtype: int64

Notice that the `Series` is created in key-sorted order.

If we pass a custom index to `Series`, it will select the corresponding values from the dict, and treat indices without corrsponding values as missing. pandas uses the `NaN` (not a number) type for missing values.

In [19]:
bacteria2 = pd.Series(bacteria_dict, 
                      index=['Cyanobacteria','Firmicutes','Proteobacteria','Actinobacteria'])
bacteria2

Cyanobacteria      NaN
Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
dtype: float64

In [20]:
bacteria2.isnull()

Cyanobacteria      True
Firmicutes        False
Proteobacteria    False
Actinobacteria    False
dtype: bool

Critically, the labels are used to **align data** when used in operations with other Series objects:

In [21]:
bacteria + bacteria2

Actinobacteria    1138
Bacteroidetes      NaN
Cyanobacteria      NaN
Firmicutes        1264
Proteobacteria    3276
dtype: float64

Contrast this with NumPy arrays, where arrays of the same length will combine values element-wise; adding Series combined values with the same label in the resulting series. Notice also that the missing values were propogated by addition.

### DataFrame

Inevitably, we want to be able to store, view and manipulate data that is *multivariate*, where for every index there are multiple fields or columns of data, often of varying data type.

A `DataFrame` is a tabular data structure, encapsulating multiple series like columns in a spreadsheet. Data are stored internally as a 2-dimensional object, but the `DataFrame` allows us to represent and manipulate higher-dimensional data.

In [22]:
data = pd.DataFrame({'value':[632, 1638, 569, 115, 433, 1130, 754, 555],
                     'patient':[1, 1, 1, 1, 2, 2, 2, 2],
                     'phylum':['Firmicutes', 'Proteobacteria', 'Actinobacteria', 
    'Bacteroidetes', 'Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes']})
data

,patient,phylum,value
0,1,Firmicutes,632
1,1,Proteobacteria,1638
2,1,Actinobacteria,569
3,1,Bacteroidetes,115
4,2,Firmicutes,433
5,2,Proteobacteria,1130
6,2,Actinobacteria,754
7,2,Bacteroidetes,555


Notice the `DataFrame` is sorted by column name. We can change the order by indexing them in the order we desire:

In [23]:
data[['phylum','value','patient']]

,phylum,value,patient
0,Firmicutes,632,1
1,Proteobacteria,1638,1
2,Actinobacteria,569,1
3,Bacteroidetes,115,1
4,Firmicutes,433,2
5,Proteobacteria,1130,2
6,Actinobacteria,754,2
7,Bacteroidetes,555,2


A `DataFrame` has a second index, representing the columns:

In [24]:
data.columns

Index(['patient', 'phylum', 'value'], dtype='object')

If we wish to access columns, we can do so either by dict-like indexing or by attribute:

In [25]:
data['value']

0     632
1    1638
2     569
3     115
4     433
5    1130
6     754
7     555
Name: value, dtype: int64

In [26]:
data.value

0     632
1    1638
2     569
3     115
4     433
5    1130
6     754
7     555
Name: value, dtype: int64

Using the standard indexing syntax for a single column of data from a `DataFrame` returns the column as a `Series`.

In [27]:
type(data['value'])

pandas.core.series.Series

Passing the column name as a list returns the column as a `DataFrame` instead.

In [28]:
data[['value']]

,value
0,632
1,1638
2,569
3,115
4,433
5,1130
6,754
7,555


Notice that indexing works differently with a `DataFrame` than with a `Series`, where in the latter, dict-like indexing retrieved a particular element (row). If we want access to a row in a `DataFrame`, we index its `ix` attribute.

In [29]:
data

,patient,phylum,value
0,1,Firmicutes,632
1,1,Proteobacteria,1638
2,1,Actinobacteria,569
3,1,Bacteroidetes,115
4,2,Firmicutes,433
5,2,Proteobacteria,1130
6,2,Actinobacteria,754
7,2,Bacteroidetes,555


In [30]:
data.ix[3]

patient                1
phylum     Bacteroidetes
value                115
Name: 3, dtype: object

Notice that since a row potentially contains different data types, the returned `Series` of values is of the generic `object` type.

If we want to create a `DataFrame` row-wise rather than column-wise, we can do so with a dict of dicts:

In [31]:
data = pd.DataFrame({0: {'patient': 1, 'phylum': 'Firmicutes', 'value': 632},
                    1: {'patient': 1, 'phylum': 'Proteobacteria', 'value': 1638},
                    2: {'patient': 1, 'phylum': 'Actinobacteria', 'value': 569},
                    3: {'patient': 1, 'phylum': 'Bacteroidetes', 'value': 115},
                    4: {'patient': 2, 'phylum': 'Firmicutes', 'value': 433},
                    5: {'patient': 2, 'phylum': 'Proteobacteria', 'value': 1130},
                    6: {'patient': 2, 'phylum': 'Actinobacteria', 'value': 754},
                    7: {'patient': 2, 'phylum': 'Bacteroidetes', 'value': 555}})

In [32]:
data

,0,1,2,3,4,5,6,7
patient,1,1,1,1,2,2,2,2
phylum,Firmicutes,Proteobacteria,Actinobacteria,Bacteroidetes,Firmicutes,Proteobacteria,Actinobacteria,Bacteroidetes
value,632,1638,569,115,433,1130,754,555


However, we probably want this transposed:

In [33]:
data = data.T
data

,patient,phylum,value
0,1,Firmicutes,632
1,1,Proteobacteria,1638
2,1,Actinobacteria,569
3,1,Bacteroidetes,115
4,2,Firmicutes,433
5,2,Proteobacteria,1130
6,2,Actinobacteria,754
7,2,Bacteroidetes,555


Its important to note that the `Series` returned when a `DataFrame` is indexed is merely a **view** on the DataFrame, and not a copy of the data itself. So you must be *cautious* when manipulating this data.

For example, let's isolate a column of our dataset by assigning it as a `Series` to a variable.

In [34]:
vals = data.value
vals

0     632
1    1638
2     569
3     115
4     433
5    1130
6     754
7     555
Name: value, dtype: object

Now, let's assign a new value to one of the elements of the `Series`.

In [35]:
vals[5] = 0
vals

0     632
1    1638
2     569
3     115
4     433
5       0
6     754
7     555
Name: value, dtype: object

However, we may not anticipate that the value in the original `DataFrame` has also been changed!

In [36]:
data

,patient,phylum,value
0,1,Firmicutes,632
1,1,Proteobacteria,1638
2,1,Actinobacteria,569
3,1,Bacteroidetes,115
4,2,Firmicutes,433
5,2,Proteobacteria,0
6,2,Actinobacteria,754
7,2,Bacteroidetes,555


We can avoid this by working with a copy when modifying subsets of the original data.

In [37]:
vals = data.value.copy()
vals[5] = 1000

data

,patient,phylum,value
0,1,Firmicutes,632
1,1,Proteobacteria,1638
2,1,Actinobacteria,569
3,1,Bacteroidetes,115
4,2,Firmicutes,433
5,2,Proteobacteria,0
6,2,Actinobacteria,754
7,2,Bacteroidetes,555


So, as we have seen, we can create or modify columns by assignment; let's put back the value we accidentally changed.

In [38]:
data.value[5] = 1130

Or, we may wish to add a column representing the year the data were collected.

In [39]:
data['year'] = 2013
data

,patient,phylum,value,year
0,1,Firmicutes,632,2013
1,1,Proteobacteria,1638,2013
2,1,Actinobacteria,569,2013
3,1,Bacteroidetes,115,2013
4,2,Firmicutes,433,2013
5,2,Proteobacteria,1130,2013
6,2,Actinobacteria,754,2013
7,2,Bacteroidetes,555,2013


But note, we cannot use the attribute indexing method to add a new column:

In [40]:
data.treatment = 1
data

,patient,phylum,value,year
0,1,Firmicutes,632,2013
1,1,Proteobacteria,1638,2013
2,1,Actinobacteria,569,2013
3,1,Bacteroidetes,115,2013
4,2,Firmicutes,433,2013
5,2,Proteobacteria,1130,2013
6,2,Actinobacteria,754,2013
7,2,Bacteroidetes,555,2013


In [41]:
data.treatment

1

When adding a column that is not a simple constant, we need to be a bit more careful. Due to pandas' auto-alignment behavior, specifying a `Series` as a new column causes its values to be added according to the `DataFrame`'s index:

In [42]:
treatment = pd.Series([0]*4 + [1]*2)

treatment

0    0
1    0
2    0
3    0
4    1
5    1
dtype: int64

In [43]:
data['treatment'] = treatment

data

,patient,phylum,value,year,treatment
0,1,Firmicutes,632,2013,0
1,1,Proteobacteria,1638,2013,0
2,1,Actinobacteria,569,2013,0
3,1,Bacteroidetes,115,2013,0
4,2,Firmicutes,433,2013,1
5,2,Proteobacteria,1130,2013,1
6,2,Actinobacteria,754,2013,NaN
7,2,Bacteroidetes,555,2013,NaN


Other Python data structures (ones without an index) need to be the same length as the `DataFrame`:

In [44]:
month = ['Jan', 'Feb', 'Mar', 'Apr']
data['month'] = month

ValueError: Length of values does not match length of index

In [45]:
data['month'] = ['Jan']*len(data)

data

,patient,phylum,value,year,treatment,month
0,1,Firmicutes,632,2013,0,Jan
1,1,Proteobacteria,1638,2013,0,Jan
2,1,Actinobacteria,569,2013,0,Jan
3,1,Bacteroidetes,115,2013,0,Jan
4,2,Firmicutes,433,2013,1,Jan
5,2,Proteobacteria,1130,2013,1,Jan
6,2,Actinobacteria,754,2013,NaN,Jan
7,2,Bacteroidetes,555,2013,NaN,Jan


We can use `del` to remove columns, in the same way `dict` entries can be removed:

In [46]:
del data['month']

data

,patient,phylum,value,year,treatment
0,1,Firmicutes,632,2013,0
1,1,Proteobacteria,1638,2013,0
2,1,Actinobacteria,569,2013,0
3,1,Bacteroidetes,115,2013,0
4,2,Firmicutes,433,2013,1
5,2,Proteobacteria,1130,2013,1
6,2,Actinobacteria,754,2013,NaN
7,2,Bacteroidetes,555,2013,NaN


Or employ the `drop` method.

In [47]:
data.drop('treatment', axis=1)

,patient,phylum,value,year
0,1,Firmicutes,632,2013
1,1,Proteobacteria,1638,2013
2,1,Actinobacteria,569,2013
3,1,Bacteroidetes,115,2013
4,2,Firmicutes,433,2013
5,2,Proteobacteria,1130,2013
6,2,Actinobacteria,754,2013
7,2,Bacteroidetes,555,2013


We can extract the underlying data as a simple `ndarray` by accessing the `values` attribute:

In [48]:
data.values

array([[1, 'Firmicutes', 632, 2013, 0.0],
       [1, 'Proteobacteria', 1638, 2013, 0.0],
       [1, 'Actinobacteria', 569, 2013, 0.0],
       [1, 'Bacteroidetes', 115, 2013, 0.0],
       [2, 'Firmicutes', 433, 2013, 1.0],
       [2, 'Proteobacteria', 1130, 2013, 1.0],
       [2, 'Actinobacteria', 754, 2013, nan],
       [2, 'Bacteroidetes', 555, 2013, nan]], dtype=object)

Notice that because of the mix of string, integer and float (and `NaN`) values, the dtype of the array is `object`. The dtype will automatically be chosen to be as general as needed to accomodate all the columns.

In [78]:
df = pd.DataFrame({'foo': [1,2,3], 'bar':[0.4, -1.0, 4.5]})

df.values, df.values.dtype

(array([[ 0.4,  1. ],
        [-1. ,  2. ],
        [ 4.5,  3. ]]), dtype('float64'))

pandas uses a custom data structure to represent the **indices** of Series and DataFrames.

In [79]:
data.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')

Index objects are immutable:

In [84]:
new_index = data.index.values
new_index[0] = 15
data.index = new_index

In [85]:
data

,patient,phylum,value,year,treatment
15,1,Firmicutes,632,2013,0
1,1,Proteobacteria,1638,2013,0
2,1,Actinobacteria,569,2013,0
3,1,Bacteroidetes,115,2013,0
4,2,Firmicutes,433,2013,1
5,2,Proteobacteria,1130,2013,1
6,2,Actinobacteria,754,2013,NaN
7,2,Bacteroidetes,555,2013,NaN


This is so that Index objects can be shared between data structures without fear that they will be changed.

In [81]:
bacteria2.index = bacteria.index

bacteria2

phylum
Firmicutes         NaN
Proteobacteria     632
Actinobacteria    1638
Bacteroidetes      569
dtype: float64

### Excercise: Indexing

From the `data` table above, create an index to return all rows for which the phylum name ends in "bacteria" and the value is greater than 1000.

In [52]:
# Write your answer here

## Importing data

A key, but often under-appreciated, step in data analysis is importing the data that we wish to analyze. Though it is easy to load basic data structures into Python using built-in tools or those provided by packages like NumPy, it is non-trivial to import structured data well, and to easily convert this input into a robust data structure:

    genes = np.loadtxt("genes.csv", delimiter=",", dtype=[('gene', '|S10'), ('value', '<f4')])

pandas provides a convenient set of functions for importing tabular data in a number of formats directly into a `DataFrame` object. These functions include a slew of options to perform type inference, indexing, parsing, iterating and cleaning automatically as data are imported.

In [54]:
medals = pd.read_table('../data/olympics.1996.txt', sep='\t',
                       index_col=0,
                       header=None, names=['country', 'medals', 'population'])
medals.head()

,medals,population
country,,
Tonga,1,96165
Bahamas,1,281584
Jamaica,6,2589043
Cuba,25,10952046
Australia,41,18348078


In [55]:
oecd = pd.read_html('http://www.oecd.org/about/membersandpartners/list-oecd-member-countries.htm', 
                    header=0)[1][[1,2]]
oecd.head()

,Country,Date
0,AUSTRALIA,7 June 1971
1,AUSTRIA,29 September 1961
2,BELGIUM,13 September 1961
3,CANADA,10 April 1961
4,CHILE,7 May 2010


In [57]:
oecd['year'] = oecd.Date.dropna().str.split().apply(lambda x: int(x[-1]))
oecd_year = oecd.set_index(oecd.Country.str.capitalize())['year'].dropna()
oecd_year

Country
Australia          1971
Austria            1961
Belgium            1961
Canada             1961
Chile              2010
Czech republic     1995
Denmark            1961
Estonia            2010
Finland            1969
France             1961
Germany            1961
Greece             1961
Hungary            1996
Iceland            1961
Ireland            1961
Israel             2010
Italy              1962
Japan              1964
Korea              1996
Luxembourg         1961
Mexico             1994
Netherlands        1961
New zealand        1973
Norway             1961
Poland             1996
Portugal           1961
Slovak republic    2000
Slovenia           2010
Spain              1961
Sweden             1961
Switzerland        1961
Turkey             1961
United kingdom     1961
United states      1961
Name: year, dtype: float64

In [58]:
medals_data = medals.assign(oecd=medals.index.isin((oecd_year<1997).index))
medals_data = medals_data.assign(log_population=lambda x: np.log(x.population))

Let's start with some more bacteria data, stored in csv format.

In [118]:
!cat ../data/microbiome/microbiome.csv

Taxon,Patient,Tissue,Stool
Firmicutes,1,632,305
Firmicutes,2,136,4182
Firmicutes,3,1174,703
Firmicutes,4,408,3946
Firmicutes,5,831,8605
Firmicutes,6,693,50
Firmicutes,7,718,717
Firmicutes,8,173,33
Firmicutes,9,228,80
Firmicutes,10,162,3196
Firmicutes,11,372,32
Firmicutes,12,4255,4361
Firmicutes,13,107,1667
Firmicutes,14,96,223
Firmicutes,15,281,2377
Proteobacteria,1,1638,3886
Proteobacteria,2,2469,1821
Proteobacteria,3,839,661
Proteobacteria,4,4414,18
Proteobacteria,5,12044,83
Proteobacteria,6,2310,12
Proteobacteria,7,3053,547
Proteobacteria,8,395,2174
Proteobacteria,9,2651,767
Proteobacteria,10,1195,76
Proteobacteria,11,6857,795
Proteobacteria,12,483,666
Proteobacteria,13,2950,3994
Proteobacteria,14,1541,816
Proteobacteria,15,1307,53
Actinobacteria,1,569,648
Actinobacteria,2,1590,4
Actinobacteria,3,25,2
Actinobacteria,4,259,300
Actinobacteria,5,568,7
Actinobacteria,6,1102,9
Actinobacteria,7,678,377
Actinobacteria,8,260,58
Actinobacteria,9,424,233
Actinobacteria,10,548,21
Actinobacteri

This table can be read into a DataFrame using `read_csv`:

In [121]:
mb = pd.read_csv("../data/microbiome/microbiome.csv")
mb

,Taxon,Patient,Tissue,Stool
0,Firmicutes,1,632,305
1,Firmicutes,2,136,4182
2,Firmicutes,3,1174,703
3,Firmicutes,4,408,3946
4,Firmicutes,5,831,8605
5,Firmicutes,6,693,50
6,Firmicutes,7,718,717
7,Firmicutes,8,173,33
8,Firmicutes,9,228,80
9,Firmicutes,10,162,3196


Notice that `read_csv` automatically considered the first row in the file to be a header row.

We can override default behavior by customizing some the arguments, like `header`, `names` or `index_col`.

In [123]:
pd.read_csv("../data/microbiome/microbiome.csv", header=None).head(3)

,0,1,2,3
0,Taxon,Patient,Tissue,Stool
1,Firmicutes,1,632,305
2,Firmicutes,2,136,4182


`read_csv` is just a convenience function for `read_table`, since csv is such a common format:

In [124]:
mb = pd.read_table("../data/microbiome/microbiome.csv", sep=',')

The `sep` argument can be customized as needed to accomodate arbitrary separators. For example, we can use a regular expression to define a variable amount of whitespace, which is (unfortunately) common in some datasets: 
    
    sep='\s+'

For a more useful index, we can specify the first two columns, which together provide a unique index to the data.

In [125]:
mb = pd.read_csv("../data/microbiome/microbiome.csv", index_col=['Taxon','Patient'])
mb.head()

Tissue  Stool
Taxon      Patient               
Firmicutes 1           632    305
           2           136   4182
           3          1174    703
           4           408   3946
           5           831   8605

In [129]:
mb.ix[('Firmicutes', 2)]

Tissue     136
Stool     4182
Name: (Firmicutes, 2), dtype: int64

This is called a *hierarchical* index, which we will revisit later in the tutorial.

If we have sections of data that we do not wish to import (for example, known bad data), we can populate the `skiprows` argument:

In [135]:
pd.read_csv("../data/microbiome/microbiome.csv", skiprows=[3,4,6]).head()

,Taxon,Patient,Tissue,Stool
0,Firmicutes,1,632,305
1,Firmicutes,2,136,4182
2,Firmicutes,5,831,8605
3,Firmicutes,7,718,717
4,Firmicutes,8,173,33


Conversely, if we only want to import a small number of rows from, say, a very large data file we can use `nrows`:

In [137]:
few_recs = pd.read_csv("../data/microbiome/microbiome.csv", nrows=4)

few_recs

,Taxon,Patient,Tissue,Stool
0,Firmicutes,1,632,305
1,Firmicutes,2,136,4182
2,Firmicutes,3,1174,703
3,Firmicutes,4,408,3946


Alternately, if we want to process our data in reasonable chunks, the `chunksize` argument will return an iterable object that can be employed in a data processing loop. For example, our microbiome data are organized by bacterial phylum, with 15 patients represented in each:

In [138]:
data_chunks = pd.read_csv("../data/microbiome/microbiome.csv", chunksize=15)

In [139]:
data_chunks

In [140]:
mean_tissue = {chunk.Taxon[0]:chunk.Tissue.mean() for chunk in data_chunks}
    
mean_tissue

{'Actinobacteria': 449.06666666666666,
 'Bacteroidetes': 599.6666666666666,
 'Firmicutes': 684.4,
 'Other': 198.8,
 'Proteobacteria': 2943.0666666666666}

Most real-world data is incomplete, with values missing due to incomplete observation, data entry or transcription error, or other reasons. pandas will automatically recognize and parse common missing data indicators, including `NA` and `NULL`.

In [141]:
!cat ../data/microbiome/microbiome_missing.csv

Taxon,Patient,Tissue,Stool
Firmicutes,1,632,305
Firmicutes,2,136,4182
Firmicutes,3,,703
Firmicutes,4,408,3946
Firmicutes,5,831,8605
Firmicutes,6,693,50
Firmicutes,7,718,717
Firmicutes,8,173,33
Firmicutes,9,228,NA
Firmicutes,10,162,3196
Firmicutes,11,372,-99999
Firmicutes,12,4255,4361
Firmicutes,13,107,1667
Firmicutes,14,?,223
Firmicutes,15,281,2377
Proteobacteria,1,1638,3886
Proteobacteria,2,2469,1821
Proteobacteria,3,839,661
Proteobacteria,4,4414,18
Proteobacteria,5,12044,83
Proteobacteria,6,2310,12
Proteobacteria,7,3053,547
Proteobacteria,8,395,2174
Proteobacteria,9,2651,767
Proteobacteria,10,1195,76
Proteobacteria,11,6857,795
Proteobacteria,12,483,666
Proteobacteria,13,2950,3994
Proteobacteria,14,1541,816
Proteobacteria,15,1307,53
Actinobacteria,1,569,648
Actinobacteria,2,1590,4
Actinobacteria,3,25,2
Actinobacteria,4,259,300
Actinobacteria,5,568,7
Actinobacteria,6,1102,9
Actinobacteria,7,678,377
Actinobacteria,8,260,58
Actinobacteria,9,424,233
Actinobacteria,10,548,21
Actinobacteria

In [142]:
pd.read_csv("../data/microbiome/microbiome_missing.csv").head(20)

,Taxon,Patient,Tissue,Stool
0,Firmicutes,1,632,305
1,Firmicutes,2,136,4182
2,Firmicutes,3,NaN,703
3,Firmicutes,4,408,3946
4,Firmicutes,5,831,8605
5,Firmicutes,6,693,50
6,Firmicutes,7,718,717
7,Firmicutes,8,173,33
8,Firmicutes,9,228,NaN
9,Firmicutes,10,162,3196


Above, pandas recognized `NA` and an empty field as missing data.

In [143]:
pd.isnull(pd.read_csv("../data/microbiome/microbiome_missing.csv")).head(20)

,Taxon,Patient,Tissue,Stool
0,False,False,False,False
1,False,False,False,False
2,False,False,True,False
3,False,False,False,False
4,False,False,False,False
5,False,False,False,False
6,False,False,False,False
7,False,False,False,False
8,False,False,False,True
9,False,False,False,False


Unfortunately, there will sometimes be inconsistency with the conventions for missing data. In this example, there is a question mark "?" and a large negative number where there should have been a positive integer. We can specify additional symbols with the `na_values` argument:
   

In [144]:
pd.read_csv("../data/microbiome/microbiome_missing.csv", na_values=['?', -99999]).head(20)

,Taxon,Patient,Tissue,Stool
0,Firmicutes,1,632,305
1,Firmicutes,2,136,4182
2,Firmicutes,3,NaN,703
3,Firmicutes,4,408,3946
4,Firmicutes,5,831,8605
5,Firmicutes,6,693,50
6,Firmicutes,7,718,717
7,Firmicutes,8,173,33
8,Firmicutes,9,228,NaN
9,Firmicutes,10,162,3196


These can be specified on a column-wise basis using an appropriate dict as the argument for `na_values`.

### Microsoft Excel

Since so much financial and scientific data ends up in Excel spreadsheets (regrettably), pandas' ability to directly import Excel spreadsheets is valuable. This support is contingent on having one or two dependencies (depending on what version of Excel file is being imported) installed: `xlrd` and `openpyxl` (these may be installed with either `pip` or `easy_install`).

Importing Excel data to pandas is a two-step process. First, we create an `ExcelFile` object using the path of the file:                                             

In [145]:
mb_file = pd.ExcelFile('../data/microbiome/MID1.xls')
mb_file

Then, since modern spreadsheets consist of one or more "sheets", we parse the sheet with the data of interest:

In [146]:
mb1 = mb_file.parse("Sheet 1", header=None)
mb1.columns = ["Taxon", "Count"]
mb1.head()

,Taxon,Count
0,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",7
1,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",2
2,"Archaea ""Crenarchaeota"" Thermoprotei Sulfoloba...",3
3,"Archaea ""Crenarchaeota"" Thermoprotei Thermopro...",3
4,"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",7


There is now a `read_excel` conveneince function in pandas that combines these steps into a single call:

In [147]:
mb2 = pd.read_excel('../data/microbiome/MID2.xls', sheetname='Sheet 1', header=None)
mb2.head()

,0,1
0,"Archaea ""Crenarchaeota"" Thermoprotei Acidiloba...",2
1,"Archaea ""Crenarchaeota"" Thermoprotei Acidiloba...",14
2,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",23
3,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",1
4,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",2


In [148]:
pd.read_excel?

### Relational Databases

If you are fortunate, your data will be stored in a database (relational or non-relational) rather than in arbitrary text files or spreadsheet. Relational databases are particularly useful for storing large quantities of *structured* data, where fields are grouped together in tables according to their relationships with one another.

pandas' `DataFrame` interacts with relational (*i.e.* SQL) databases, and even provides facilties for using SQL syntax on the `DataFrame` itself, which we will get to later. For now, let's work with a ubiquitous embedded database called **SQLite**, which comes bundled with Python. A SQLite database can be queried with the standard library's `sqlite3` module.

In [149]:
import sqlite3

query = '''
CREATE TABLE samples
(taxon VARCHAR(15), patient INTEGER, tissue INTEGER, stool INTEGER);
'''

This query string will create a table to hold some of our microbiome data, which we can execute after connecting to a database (which will be created, if it does not exist).

In [150]:
con = sqlite3.connect('microbiome.sqlite3')
con.execute(query)
con.commit()

In [151]:
few_recs.ix[0]

Taxon      Firmicutes
Patient             1
Tissue            632
Stool             305
Name: 0, dtype: object

In [152]:
con.execute('INSERT INTO samples VALUES(\'{}\',{},{},{})'.format(*few_recs.ix[0]))

In [153]:
query = 'INSERT INTO samples VALUES(?, ?, ?, ?)'
con.executemany(query, few_recs.values[1:])

In [154]:
con.commit()

Using `SELECT` queries, we can read from the database.

In [155]:
cursor = con.execute('SELECT * FROM samples')
rows = cursor.fetchall()

rows

[('Firmicutes', 1, 632, 305),
 ('Firmicutes', 2, 136, 4182),
 ('Firmicutes', 3, 1174, 703),
 ('Firmicutes', 4, 408, 3946)]

These results can be passed directly to a `DataFrame`

In [156]:
pd.DataFrame(rows)

,0,1,2,3
0,Firmicutes,1,632,305
1,Firmicutes,2,136,4182
2,Firmicutes,3,1174,703
3,Firmicutes,4,408,3946


To obtain the column names, we can obtain the table information from the database, via the special `PRAGMA` statement.

In [157]:
table_info = con.execute('PRAGMA table_info(samples);').fetchall()

table_info

[(0, 'taxon', 'VARCHAR(15)', 0, None, 0),
 (1, 'patient', 'INTEGER', 0, None, 0),
 (2, 'tissue', 'INTEGER', 0, None, 0),
 (3, 'stool', 'INTEGER', 0, None, 0)]

In [158]:
pd.DataFrame(rows, columns=np.transpose(table_info)[1])

,taxon,patient,tissue,stool
0,Firmicutes,1,632,305
1,Firmicutes,2,136,4182
2,Firmicutes,3,1174,703
3,Firmicutes,4,408,3946


A more direct approach is to pass the query to the `read_sql_query` functon, which returns a populated `DataFrame.

In [163]:
pd.read_sql_query('SELECT * FROM samples', con)

,taxon,patient,tissue,stool
0,Firmicutes,1,632,305
1,Firmicutes,2,136,4182
2,Firmicutes,3,1174,703
3,Firmicutes,4,408,3946


Correspondingly, we can append records into the database with `to_sql`.

In [164]:
more_recs = pd.read_csv("../data/microbiome/microbiome_missing.csv").head(20)

In [165]:
more_recs.to_sql('samples', con, if_exists='append', index=False)

In [166]:
cursor = con.execute('SELECT * FROM samples')
cursor.fetchall()

[('Firmicutes', 1, 632, 305),
 ('Firmicutes', 2, 136, 4182),
 ('Firmicutes', 3, 1174, 703),
 ('Firmicutes', 4, 408, 3946),
 ('Firmicutes', 1, 632, 305),
 ('Firmicutes', 2, 136, 4182),
 ('Firmicutes', 3, None, 703),
 ('Firmicutes', 4, 408, 3946),
 ('Firmicutes', 5, 831, 8605),
 ('Firmicutes', 6, 693, 50),
 ('Firmicutes', 7, 718, 717),
 ('Firmicutes', 8, 173, 33),
 ('Firmicutes', 9, 228, None),
 ('Firmicutes', 10, 162, 3196),
 ('Firmicutes', 11, 372, -99999),
 ('Firmicutes', 12, 4255, 4361),
 ('Firmicutes', 13, 107, 1667),
 ('Firmicutes', 14, '?', 223),
 ('Firmicutes', 15, 281, 2377),
 ('Proteobacteria', 1, 1638, 3886),
 ('Proteobacteria', 2, 2469, 1821),
 ('Proteobacteria', 3, 839, 661),
 ('Proteobacteria', 4, 4414, 18),
 ('Proteobacteria', 5, 12044, 83)]

There are several other data formats that can be imported into Python and converted into DataFrames, with the help of buitl-in or third-party libraries. These include JSON, XML, HDF5, non-relational databases, and various web APIs.

In [167]:
# Get rid of the database we created
!rm microbiome.sqlite3

# Using pandas

This section, we will import and clean up some of the datasets that we will be using later on in the tutorial. And in doing so, we will introduce the key functionality of pandas that is required to use the software effectively.

### Manipulating indices

**Reindexing** allows users to manipulate the data labels in a DataFrame. It forces a DataFrame to conform to the new index, and optionally, fill in missing data if requested.

A simple use of `reindex` is to alter the order of the rows:

Notice that the reindexing operation is not performed "in-place"; the original `DataFrame` remains as it was, and the method returns a copy of the `DataFrame` with the new index. This is a common trait for pandas, and is a Good Thing.

You can see that the `id` index is not exactly sequential -- some index numbers are missing. Say we wanted to populate the table with every `id` value. We could specify and index that is a sequence from the first to the last `id` numbers in the database, and pandas would fill in the missing data with `NaN` values:

Missing values can be filled as desired, either with selected values, or by rule:

Keep in mind that `reindex` does not work if we pass a non-unique index series.

We can remove rows or columns via the `drop` method:

## Indexing and Selection

Indexing works analogously to indexing in NumPy arrays, except we can use the labels in the `Index` object to extract values in addition to arrays of integers.

We can also slice with data labels, since they have an intrinsic order within the Index:

In a `DataFrame` we can slice along either or both axes:

Similarly, the cross-section method `xs` (not a field) extracts a single column or row *by label* and returns it as a `Series`:

## Operations

`DataFrame` and `Series` objects allow for several operations to take place either on a single object, or between two or more objects.

For example, we can perform arithmetic on the elements of two objects, such as combining baseball statistics across years:

pandas' data alignment places `NaN` values for labels that do not overlap in the two Series. In fact, there are only 6 players that occur in both years.

While we do want the operation to honor the data labels in this way, we probably do not want the missing values to be filled with `NaN`. We can use the `add` method to calculate player home run totals by using the `fill_value` argument to insert a zero for home runs where labels do not overlap:

We can also apply functions to each column or row of a `DataFrame`

## Categorical Data

Recently introduced in pandas (v. 0.15), the `category` data type is meant to represent **categorical** variables. These are variables that can take on a finite set of pre-defined values. This can include things like the genus of an organism (as we have seen above), country, race, gender, or other demographic information for which the possible values is limited to a countable set. 

The `cut` function in pandas is used to generate discrete categories from continuous data, according to a set of specified bins. If we pass labels to `cut`, it will return the resulting data as `category`.

It is important to differentiate the pandas' `category` type from the stricter statistical definition of a categorical variable. In the latter, the categories are always unordered (*contra* ordinal variables), whereas the `category` data type may be unordered or ordered. In the home runs example above, the categories are ordered, and the `cut` function infers these automatically. In the bacteria example, the label names are unordered.

However, categories may be arbitrarily ordered, as required.

Though categorical data may have an order, they do not support arithmetic operations. Order is defined by the order of categories, not lexical order of the values. Internally, the data structure consists of a categories array and an integer array of codes which point to the real value in the categories array.

All values of categorical data are either in categories or `np.nan`.

The `from_codes()` function can be used to apply existing category names to the construction of a categorical variable.

For example, perhaps we would like to turn the `patient` field in the microbiome dataset into a categorical variable, using the patient names as labels.

In [258]:
data_patient = pd.Categorical.from_codes(data.patient-1, categories=['Chris', 'Skipper'])

## Hierarchical indexing

In the baseball example, I was forced to combine 3 fields to obtain a unique index that was not simply an integer value. A more elegant way to have done this would be to create a **hierarchical index** from the three fields.

This index is a `MultiIndex` object that consists of a sequence of tuples, the elements of which is some combination of the three columns used to create the index. Where there are multiple repeated values, pandas does not print the repeats, making it easy to identify groups of values.

Rows can be indexed by passing the appropriate tuple.

With a hierachical index, we can select subsets of the data based on a *partial* index:

In [298]:
mb.ix['Proteobacteria']

Tissue     136
Stool     4182
Name: (Firmicutes, 2), dtype: int64

In [ ]:
mb.xs(1, level='patient')

In [303]:
mb.swaplevel('Patient', 'Taxon').head()

,,Tissue,Stool
Patient,Taxon,,
1,Firmicutes,632,305
2,Firmicutes,136,4182
3,Firmicutes,1174,703
4,Firmicutes,408,3946
5,Firmicutes,831,8605


## Missing data

The occurence of missing data is so prevalent that it pays to use tools like pandas, which seamlessly integrates missing data handling so that it can be dealt with easily, and in the manner required by the analysis at hand.

Missing data are represented in `Series` and `DataFrame` objects by the `NaN` floating point value. However, `None` is also treated as missing, since it is commonly used as such in other contexts (*e.g.* NumPy).

By default, `dropna` drops entire rows in which one or more values are missing.

If we want to drop missing values column-wise instead of row-wise, we use `axis=1`.

Rather than omitting missing data from an analysis, in some cases it may be suitable to fill the missing value in, either with a default value (such as zero) or a value that is either imputed or carried forward/backward from similar data points. We can do this programmatically in pandas with the `fillna` argument.

## Writing Data to Files

As well as being able to read several data input formats, pandas can also export data to a variety of storage formats. We will bring your attention to just a couple of these.

In [345]:
mb.to_csv("../data/mb.csv", index=False)

The `to_csv` method writes a `DataFrame` to a comma-separated values (csv) file. You can specify custom delimiters (via `sep` argument), how missing values are written (via `na_rep` argument), whether the index is writen (via `index` argument), whether the header is included (via `header` argument), among other options.

## Exercise: Compiling Ebola Data

The `../data/ebola` folder contains summarized reports of Ebola cases from three countries during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

From these data files, use pandas to import them and create a single data frame that includes the daily totals of new cases and deaths for each country. 

We may use this compiled data for more advaned applications later in the course.

In [ ]:
# Write your answer here